In [2]:
import numpy as np
from numpy.linalg import inv as inv

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

def svt(mat, tau):
    u, s, v = np.linalg.svd(mat, full_matrices = False)
    vec = s - tau
    vec[vec < 0] = 0
    return np.matmul(np.matmul(u, np.diag(vec)), v)

In [3]:
def svt_tnn(mat, alpha, rho, theta):
    """This is a Numpy dependent singular value thresholding (SVT) process."""
    u, s, v = np.linalg.svd(mat, full_matrices = False)
    vec = s.copy()
    vec[theta :] = s[theta :] - alpha / rho
    vec[vec < 0] = 0
    return np.matmul(np.matmul(u, np.diag(vec)), v)

In [4]:
def svt_tnn(mat, alpha, rho, theta):
    tau = alpha / rho
    [m, n] = mat.shape
    if 2 * m < n:
        u, s, v = np.linalg.svd(mat @ mat.T, full_matrices = False)
        s = np.sqrt(s)
        idx = np.sum(s > tau)
        mid = np.zeros(idx)
        mid[:theta] = 1
        mid[theta:idx] = (s[theta : idx] - tau) / s[theta : idx]
        return (u[:,:idx] @ np.diag(mid)) @ (u[:, :idx].T @ mat)
    elif m > 2 * n:
        return svt_tnn(mat.T, tau, theta).T
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    idx = np.sum(s > tau)
    vec = s[:idx].copy()
    vec[theta : idx] = s[theta : idx] - tau
    return u[:, :idx] @ np.diag(vec) @ v[:idx, :]

In [5]:
def LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter):
    """Low-Rank Tenor Completion with Truncated Nuclear Norm, LRTC-TNN."""
    dim = np.array(sparse_tensor.shape)
    pos_missing = np.where(sparse_tensor == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    
    X = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{X}}
    T = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{T}}
    Z = sparse_tensor.copy()
    last_tensor = sparse_tensor.copy()
    snorm = np.sqrt(np.sum(sparse_tensor ** 2))
    it = 0
    while True:
        rho = min(rho * 1.05, 1e5)
        for k in range(len(dim)):
            X[k] = mat2ten(svt_tnn(ten2mat(Z - T[k] / rho, k), alpha[k], rho, np.int(np.ceil(theta * dim[k]))), dim, k)
        Z[pos_missing] = np.mean(X + T / rho, axis = 0)[pos_missing]
        T = T + rho * (X - np.broadcast_to(Z, np.insert(dim, 0, len(dim))))
        tensor_hat = np.einsum('k, kmnt -> mnt', alpha, X)
        tol = np.sqrt(np.sum((tensor_hat - last_tensor) ** 2)) / snorm
        last_tensor = tensor_hat.copy()
        it += 1
        print('Iter: {}'.format(it))
        print('Tolerance: {:.6}'.format(tol))
        print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
        print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
        print()
        if (tol < epsilon) or (it >= maxiter):
            break
    
    print('Total iteration: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat

In [6]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [7]:
def HaLRTC_imputer(dense_tensor, sparse_tensor, alpha: list, rho: float, epsilon: float, maxiter: int):
    dim = np.array(sparse_tensor.shape)
    print(dim)
    tensor_hat = sparse_tensor
    pos_missing = np.where(sparse_tensor == 0.0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    B = [np.zeros(sparse_tensor.shape) for _ in range(len(dim))]
    Y = [np.zeros(sparse_tensor.shape) for _ in range(len(dim))]
    last_ten = sparse_tensor.copy()
    snorm = np.linalg.norm(sparse_tensor)
    
    it = 0
    while True:
        for k in range(len(dim)):
            B[k] = mat2ten(svt(ten2mat(tensor_hat + Y[k] / rho, k), alpha[k] / rho), dim, k)
        tensor_hat[pos_missing] = ((sum(B) - sum(Y) / rho) / 3)[pos_missing]
        for k in range(len(dim)):
            Y[k] = Y[k] - rho * (B[k] - tensor_hat)
        tol = np.linalg.norm((tensor_hat - last_ten)) / snorm
        last_ten = tensor_hat.copy()
        it += 1
        if it % 1 == 0:
            print('Iter: {}'.format(it))
            print('Tolerance: {:.6}'.format(tol))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
            print()
        if (it>3) and ((tol < epsilon) or (it >= maxiter)):
            break
    
    print('Total iteration: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat

In [8]:
def compute_tensor(X, Y, Z, n, m):
        nx, ny, nz = n
        ten = np.zeros(nx*ny*nz);
        for i in range(m):
            ten += np.kron(X[i], np.kron(Y[i], Z[i]))
        return ten;

In [9]:
n = (150, 80, 150)
nx, ny, nz = n
r = 70
V_x = np.random.normal(0,1,(r,nx))
V_y = np.random.normal(0,1,(r,ny))
V_z = np.random.normal(0,1,(r,nz))

In [10]:
dense_tensor = compute_tensor(V_x, V_y, V_z, n, r).reshape(n)
missing_rate = 0.9
random_tensor = np.random.rand(*dense_tensor.shape)
    
### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

import time
start = time.time()
alpha = [1/3, 1/3, 1/3]
rho = 0.01
theta = 0.05
epsilon = 1e-5
maxiter = 100
#tensor_hat = HaLRTC_imputer(dense_tensor, sparse_tensor, alpha, rho, epsilon, maxiter)
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
print(np.sqrt(np.sum(tensor_hat-dense_tensor)**2))
end = time.time()
print('Running time: %.2f minutes'% ((end - start)/60.0))

Iter: 1
Tolerance: 0.0923078
MAPE: 0.00475462
RMSE: 8.50868

Iter: 2
Tolerance: 0.0924807
MAPE: 0.00833959
RMSE: 8.49352

Iter: 3
Tolerance: 0.0142424
MAPE: 0.0130502
RMSE: 8.4802

Iter: 4
Tolerance: 0.0132352
MAPE: 0.0167369
RMSE: 8.46728

Iter: 5
Tolerance: 0.0118645
MAPE: 0.0207326
RMSE: 8.45558

Iter: 6
Tolerance: 0.0108607
MAPE: 0.0245736
RMSE: 8.44448

Iter: 7
Tolerance: 0.00977671
MAPE: 0.0282958
RMSE: 8.43427

Iter: 8
Tolerance: 0.00890144
MAPE: 0.031912
RMSE: 8.42468

Iter: 9
Tolerance: 0.00809735
MAPE: 0.0352871
RMSE: 8.41578

Iter: 10
Tolerance: 0.00745897
MAPE: 0.0384983
RMSE: 8.40745

Iter: 11
Tolerance: 0.00690595
MAPE: 0.041402
RMSE: 8.39969

Iter: 12
Tolerance: 0.00645814
MAPE: 0.0442321
RMSE: 8.39242

Iter: 13
Tolerance: 0.00602951
MAPE: 0.0467947
RMSE: 8.38562

Iter: 14
Tolerance: 0.0056615
MAPE: 0.0492771
RMSE: 8.37925

Iter: 15
Tolerance: 0.00529875
MAPE: 0.0515433
RMSE: 8.37328

Iter: 16
Tolerance: 0.00497769
MAPE: 0.0537093
RMSE: 8.36767

Iter: 17
Tolerance: 0.004